# Biotech Agent Guide

This notebook documents the process of testing and deploying the Biotech Agent using the Agent Development Kit (ADK).

### Objectives
1.  **Environment Setup**: Clone repo, OAuth, venv, APIs, Service Accounts.
2.  **MCP Servers Deployment**: Deploy dependent MCP servers to Cloud Run.
3.  **Local Testing**: Use `adk run` to verify the agent locally (from cloned dir).
4.  **Deployment**: Deploy the agent to Vertex AI Agent Engine.
5.  **Verification**: Verify the deployed agent using the Vertex AI SDK.
6.  **Register with Gemini Enterprise**: (Optional) Register the agent with Gemini Enterprise.
7.  **Cleanup**: Delete deployed resources.

## 1. Setup Environment

In [ ]:
# 1.0 Clone the Repository
!git clone https://github.com/seoeunbae/biotech-agent-ai.git
# git clone 부분 import로 변경하기
%cd biotech-agent-ai

In [ ]:
# # 1.0.1 Create .env file
# # Ensure you have the correct Project ID and Location
project_id = "medical-agent-481506" # Modify this to your project ID
location = "us-central1" # @param {type:"string"}

env_content = f"""GOOGLE_CLOUD_PROJECT={project_id}
GOOGLE_CLOUD_LOCATION={location}
"""

with open(".env", "w") as f:
    f.write(env_content)

print(".env file created with:")
print(env_content)

In [ ]:
# 1.1 Authenticate with Google Cloud
!gcloud auth application-default login
!gcloud config set project {project_id}
!gcloud config get-value project
!gcloud auth list
!export PROJECT_ID={project_id}

In [ ]:
# 1.2 Enable Required APIs
# Enabling APIs for Cloud Run, Artifact Registry, Cloud Build, Vertex AI, Logging, Trace, and Monitoring.
!gcloud services enable run.googleapis.com \
    aiplatform.googleapis.com \
    logging.googleapis.com \
    cloudbuild.googleapis.com \
    artifactregistry.googleapis.com \
    cloudtrace.googleapis.com \
    monitoring.googleapis.com \
    discoveryengine.googleapis.com

### 1.3 Python Virtual Environment Setup (Instructions)

To isolate dependencies, it is recommended to use a virtual environment. Run the following commands **in your terminal**:


In [ ]:

# Create virtual environment
!sudo apt update
!sudo apt install python3.12-venv
!python3 -m venv .venv

# Activate virtual environment
!source .venv/bin/activate

# 1.4 Install Dependencies in current notebook kernel (if not using venv from terminal)
!pip install google-cloud-aiplatform --upgrade --quiet
!pip install -r biotech_agent/requirements.txt

In [ ]:
import os
import sys
from google.cloud import aiplatform
import vertexai

# Configuration - Update these if needed
PROJECT_ID = "medical-agent-481506" # Modify this into your own project ID
REGION = "us-central1"
STAGING_BUCKET = "gs://txgemma_deploy_bucket_jwh" # Modify this into your own staging bucket

# Initialize Vertex AI
vertexai.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

print(f"Project: {PROJECT_ID}")
print(f"Region: {REGION}")
print(f"Staging Bucket: {STAGING_BUCKET}")

In [ ]:
# Example: Create a service account
!gcloud iam service-accounts create biotech-agent-sa --display-name="Biotech Agent Service Account"
# Grant necessary roles (e.g., Vertex AI User, Cloud Run Invoker, Log Writer)
!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member="serviceAccount:biotech-agent-sa@$PROJECT_ID.iam.gserviceaccount.com" \
    --role="roles/aiplatform.user"

!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member="serviceAccount:biotech-agent-sa@$PROJECT_ID.iam.gserviceaccount.com" \
    --role="roles/run.invoker"

!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member="serviceAccount:biotech-agent-sa@$PROJECT_ID.iam.gserviceaccount.com" \
    --role="roles/logging.logWriter"

## 2. MCP Servers Deployment

The Biotech Agent relies on several MCP servers. We need to deploy them to Cloud Run.
Note: `MCP_servers/keys` file containing API keys might be needed for some servers. Please ensure you have valid API keys from the respective data providers (OpenFDA, NCBI, etc.).

In [ ]:
# 2.1 Deploy GeneOntology MCP Server
!gcloud run deploy gene-ontology-mcp-server \
  --source biotech_agent_demo/MCP_servers/GeneOntology-MCP-Server \
  --region us-central1 \
  --project {PROJECT_ID} \
  --allow-unauthenticated

In [ ]:
# 2.2 Deploy OpenFDA MCP Server
# Note: Ensure you have your FDA_API_KEY. It may be in MCP_servers/keys
# You can source the keys file or pass it as an environment variable.
# Here we assume the source has the necessary setup or we pass it via env vars if needed.
!gcloud run deploy openfda-mcp-server \
  --source biotech_agent_demo/MCP_servers/OpenFDA-MCP-Server \
  --region us-central1 \
  --project {PROJECT_ID} \
  --allow-unauthenticated

In [ ]:
# 2.3 Deploy OpenGenes MCP Server
!gcloud run deploy opengenes-mcp \
  --source biotech_agent_demo/MCP_servers/opengenes-mcp \
  --region us-central1 \
  --project {PROJECT_ID} \
  --allow-unauthenticated

In [ ]:
# 2.4 Deploy OpenTargets MCP Server
!gcloud run deploy opentargets-mcp \
  --source biotech_agent_demo/MCP_servers/opentargets-mcp \
  --region us-central1 \
  --project {PROJECT_ID} \
  --allow-unauthenticated

In [ ]:
# 2.5 Update Agent Configuration with Deployed MCP URLs
# This step retrieves the URLs of the deployed Cloud Run services and updates the agent code.

import subprocess
import re

def get_service_url(service_name, project_id, region):
    try:
        cmd = [
            "gcloud", "run", "services", "describe", service_name,
            "--platform", "managed",
            "--region", region,
            "--project", project_id,
            "--format", "value(status.url)"
        ]
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        return result.stdout.strip()
    except subprocess.CalledProcessError as e:
        print(f"Error retrieving URL for {service_name}: {e.stderr}")
        return None

# Get URLs
print("Retrieving MCP Server URLs...")
gene_ontology_url = get_service_url("gene-ontology-mcp-server", PROJECT_ID, REGION)
openfda_url = get_service_url("openfda-mcp-server", PROJECT_ID, REGION)
opengenes_url = get_service_url("opengenes-mcp", PROJECT_ID, REGION)
opentargets_url = get_service_url("opentargets-mcp", PROJECT_ID, REGION)

# Append /sse to the URLs
gene_ontology_sse = f"{gene_ontology_url}/sse" if gene_ontology_url else None
openfda_sse = f"{openfda_url}/sse" if openfda_url else None
opengenes_sse = f"{opengenes_url}/sse" if opengenes_url else None
opentargets_sse = f"{opentargets_url}/sse" if opentargets_url else None

print(f"Gene Ontology: {gene_ontology_sse}")
print(f"OpenFDA: {openfda_sse}")
print(f"OpenGenes: {opengenes_sse}")
print(f"OpenTargets: {opentargets_sse}")

GENE_ONTOLOGY_MCP_URL="https://gene-ontology-mcp-server-520634294170.us-central1.run.app/sse"
OPENGENES_MCP_URL="https://opengenes-mcp-520634294170.us-central1.run.app/sse"
OPENFDA_MCP_URL="https://openfda-mcp-server-520634294170.us-central1.run.app/sse"
OPENTARGETS_MCP_URL="https://opentargets-mcp-520634294170.us-central1.run.app/sse"

def update_file(filepath, variable_name, new_url):
    if not new_url:
        print(f"Skipping {variable_name} (No URL found)")
        return

    try:
        with open(filepath, "r") as f:
            content = f.read()

        # Regex to replace the URL
        pattern = f'{variable_name}*=*"[^"]+"'
        replacement = f'{variable_name} = "{new_url}"'

        if re.search(pattern, content):
            new_content = re.sub(pattern, replacement, content)
            with open(filepath, "w") as f:
                f.write(new_content)
            print(f"Updated {variable_name} in {filepath}")
        else:
            print(f"Variable {variable_name} not found in {filepath}")
    except FileNotFoundError:
        print(f"File not found: {filepath}")

# Update Agent Files
print("Updating Agent Files...")

# Gene Analysis Agent
update_file(
    "biotech_agent_demo/biotech_agent/subagents/gene_analysis/agent.py",
    GENE_ONTOLOGY_MCP_URL,
    gene_ontology_sse
)
update_file(
    "biotech_agent_demo/biotech_agent/subagents/gene_analysis/agent.py",
    OPENGENES_MCP_URL,
    opengenes_sse
)

# Insight Synthesis Agent
update_file(
    "biotech_agent_demo/biotech_agent/subagents/insight_synthesis/agent.py",
    OPENFDA_MCP_URL,
    openfda_sse
)

# Normalization Agent
update_file(
    "biotech_agent_demo/biotech_agent/subagents/normalization/agent.py",
    OPENTARGETS_MCP_URL,
    opentargets_sse
)


In [ ]:
# 2.6 Verify MCP Server Deployment
# Verify that each MCP server is reachable and listing tools.

import asyncio
from mcp.client.sse import sse_client
from mcp.client.session import ClientSession

async def verify_mcp(name, sse_url):
    if not sse_url:
        print(f"Skipping verification for {name} (No URL)")
        return

    print(f"Verifying {name} at {sse_url}...")
    try:
        async with sse_client(sse_url) as (read, write):
            async with ClientSession(read, write) as session:
                await session.initialize()
                tools = await session.list_tools()
                tool_names = [t.name for t in tools.tools]
                print(f"\u2705 {name} Connected! Tools: {tool_names}")
    except Exception as e:
        print(f"\u274C {name} Failed: {e}")

print("Starting Verification...")
# Note: Running await in a notebook cell usually works if the environment supports top-level await.
# If not, use asyncio.run(verify_mcp(...)) inside a main function, but be careful with existing loops.
await verify_mcp("Gene Ontology", gene_ontology_sse)
await verify_mcp("OpenFDA", openfda_sse)
await verify_mcp("OpenGenes", opengenes_sse)
await verify_mcp("OpenTargets", opentargets_sse)


## 3. Local Testing

We use the `adk run` command to execute the agent locally. This is useful for debugging and quick verification cycles.

In [ ]:
# Test the agent with a sample query
# Ensure we are in the cloned directory
import os
if os.path.basename(os.getcwd()) != 'biotech_agent_demo':
    %cd biotech_agent_demo


## 4. Deployment to Vertex AI Agent Engine

We use `adk deploy agent_engine` to deploy the agent. This command packages the agent and deploys it as a Reasoning Engine.

In [ ]:
# Deploy to Agent Engine
# Note: ensuring we are in the root directory
import os
from dotenv import load_dotenv

os.chdir('/')
root_dir = "/content/biotech_agent_demo" # Modify this if your root directory is different
print(os.getenv("OPENTARGETS_MCP_URL"))
if os.getcwd() != root_dir and os.path.exists(root_dir):
    os.chdir(root_dir)

deploy_command = f"""
    adk deploy agent_engine biotech_agent \
      --staging_bucket {STAGING_BUCKET} \
      --region {REGION} \
      --project {PROJECT_ID} \
      --display_name=bio_research_agent
    """

print("Executing deployment command:", deploy_command)
get_ipython().system(deploy_command)


## 5. Verification

After deployment, we verify the agent functionality using the `vertexai` SDK.

In [ ]:
from vertexai.preview import reasoning_engines
import requests
from google.auth import default
from google.auth.transport.requests import Request

# Option 1: Using the ReasoningEngine client
# We list recent engines to find the one we just deployed
print("Listing Reasoning Engines...")
engines = list(reasoning_engines.ReasoningEngine.list())

# Filter for our agent if needed, or take the latest
if engines:
    latest_engine = engines[0]
    print(f"Found Agent Engine: {latest_engine.resource_name}")

    # Test Query
    query = "BRCA1 유전자의 기능은?"
    print(f"Querying: {query}")

    try:
        response = latest_engine.async_query(input={"message": query})
        print("Response:", response)
    except Exception as e:
        print(f"Query failed: {e}")
else:
    print("No Agent Engines found. Please check deployment.")

## 6. Register with Gemini Enterprise

To make your agent available in Gemini Enterprise, follow the registration guide:
[Authorize your ADK agent](https://docs.cloud.google.com/gemini/enterprise/docs/register-and-manage-an-adk-agent#authorize-your-adk-agent)

### 6.1 Create OAuth Client ID
Follow the documentation to create an OAuth Client ID for your agent.

### 6.2 Register Agent with Gemini Enterprise
Use the Agent Engine resource name (from step 4 or 5) to register your agent.

## 7. Cleanup

To avoid incurring charges, delete the deployed resources:

In [ ]:
# 7.1 Delete Agent Engine
from vertexai.preview import reasoning_engines
engines = reasoning_engines.ReasoningEngine.list()
for engine in engines:
    if engine.display_name == "bio_research_agent":
        print(f"Deleting Agent Engine: {engine.resource_name}")
        engine.delete()


In [ ]:
# 7.2 Delete Cloud Run Services (MCP Servers)
services = [
    "gene-ontology-mcp-server",
    "openfda-mcp-server",
    "opengenes-mcp",
    "opentargets-mcp"
]

for service in services:
    print(f"Deleting Cloud Run service: {service}")
    !gcloud run services delete {service} --region {REGION} --project {PROJECT_ID} --quiet

In [ ]:
# 7.3 Delete Artifact Registry Images
# Note: This deletes images associated with the services.
# Be careful if you share repositories.
# Assuming default repository for Cloud Run deployments or specific ones if created.

repos = ["cloud-run-source-deploy"]
for repo in repos:
    print(f"Listing images in repository: {repo}")
    !gcloud artifacts docker images list {REGION}-docker.pkg.dev/{PROJECT_ID}/{repo} --format="value(package)" > images_to_delete.txt

    # Optional: logic to delete specific images if needed.
    # For safety, we print instructions rather than auto-deleting all images in a shared repo.
    print(f"To delete images in {repo}, you can use:")
    print(f"gcloud artifacts docker images delete {REGION}-docker.pkg.dev/{PROJECT_ID}/{repo}/<IMAGE_NAME> --delete-tags --quiet")
